In [97]:
import json
import pandas
import numpy as np

!smt export
with open('.smt/records_export.json') as ff:
    data = json.load(ff)
    
with open('record1.json', 'w') as record1:
    for entry in range(len(data)):
        record1.write(json.dumps(data[entry], sort_keys=True, indent=4, separators=(',', ': ')))

df = pandas.DataFrame(data)

In [128]:
import glob
import os

df['output_data'][0][-2]['path']
labels = []

labels = df['label']
print labels

dfC = pandas.DataFrame(columns=['label','f','steps','time', 'N', 'cvar'])

def extractor(labelz):  #will work as long as there is 1 .mpz.npz file in the simulation directory
    rows = 0
    for lbl in labelz:
        dictt = {}
        dictt.update({'label':lbl})
        datapath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', lbl)
        mfile = glob.glob('{0}/*.mpz.npz'.format(datapath))
#         print mfile
        for flz in mfile:
            fn = np.load(flz)
            for item in fn:
                dictt.update({str(item):fn[item]})
#             print dictt
            dfC.loc[rows]=dictt
            
            
        
# extractor(labels)

# dfC

0    48841eea4a91
1    da25c86b72f0
2    bf22b1bfaef6
3    7f52183544f2
4    47235abe3a7b
Name: label, dtype: object
[]
[]
[]
['/data/aem1/new1a/surf-research/Andrey/Data/7f52183544f2/1a10step32.mpz.npz']
{'cvar': array([ 0.51891189,  0.52142702,  0.52280435,  0.52252376,  0.52069104,
        0.51799953,  0.51547484,  0.5140862 ,  0.51436936,  0.51621498,
        0.52218654,  0.52470419,  0.52608155,  0.52579918,  0.52396339,
        0.52126785,  0.51873938,  0.51734927,  0.51763456,  0.51948492,
        0.52415736,  0.5266735 ,  0.52804966,  0.5277673 ,  0.52593262,
        0.52323838,  0.5207107 ,  0.51932092,  0.51960641,  0.52145664,
        0.52408197,  0.52659408,  0.52796882,  0.52768814,  0.52585763,
        0.52316849,  0.5206449 ,  0.51925662,  0.51954039,  0.52138637,
        0.52198872,  0.52449576,  0.52586934,  0.5255914 ,  0.52376657,
        0.52108446,  0.51856678,  0.5171806 ,  0.51746141,  0.51930059,
        0.51866652,  0.5211684 ,  0.522541  ,  0.52226591,  0.5204

,label,f,steps,time,N,cvar
0,7f52183544f2,0.786513270533,32,1.0,10,"[0.518911886971, 0.521427016706, 0.52280435362..."


In [98]:
df
results = {}
list = {}
fn = np.load('/data/aem1/new1a/surf-research/Andrey/Data/47235abe3a7b/1a10x1step20.npz')

for x in fn:
        print x
# fn['time']
# del results
# print results

N
cvar
f
steps
time


In [ ]:
import os
import numpy as np

results = {}

#This code will access every step file saved in the Data/[label] directory 
for label in df['label']:
    filepath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', label)
    #gets the last step file in the directory (hopefully at equillibrium)
    file = os.listdir('Data/{0}'.format(label))[len(os.listdir('Data/{0}'.format(label)))-2] 
    if file == 'memory_usage.txt':
        break #the memory file should always be the last one saved
    else:
        fn = np.load(os.path.join(filepath, file))
        print file
        size = file[2 : file.find('s')] #the grid size based on filename
        
#         appender = {}
        #for every simulation time, create a dictionary entry to match the cvar values at that time
#         for x in xrange(len(fn['time'])):
#             appender.update({fn['time'][x]:fn['c_var'][x]})
            
        
        
        print fn["time"]

        #add a dictionary entry to associate the size of the grids with the dictionaries of times and their cvar outputs
#         results.update({size:appender})
        data = {'cvar':fn['cvar'], 'freeEn':float(fn['f']), 'step':int(fn['steps']), 'elapsed':float(fn['time'])}
#         print type(data)
#         print type(str(fn['grid']))
        results.update({str(fn['grid']):data})
        
print results
#now we can load this dictionary into a dataframe or interpolate

In [36]:
results['10x1']['freeEn']

0.7865168606271677

In [30]:
print len(df['output_data'][0])

4


In [31]:
for p in 'N', 'steps', 'sweeps':
    df[p] = df.parameters.map(lambda x: json.loads(x['content'])[p])

In [ ]:
print df['parameters'][0]['content']

In [37]:
df['Free Energy']=results['10x1']['freeEn']
df['elapsed']=results['10x1']['elapsed']

columns.append('Free Energy')
columns.append('elapsed')
df[columns]

,label,N,steps,tags,duration,sweeps,memory,Free Energy,elapsed
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0,0.786517,0.57275


In [33]:
columns = ['label', 'N', 'steps', 'tags', 'duration', 'sweeps']
print df[columns]

          label   N  steps       tags  duration  sweeps
0  8f1f49c60b96  10     20  [npzTest]  4.042712       2


In [34]:
import numpy as np
import os
num = 0
memoryArray = np.array([])
for num in range(len(df.index)):
    memName = df['output_data'][num][len(df['output_data'][num])-1]['path']
    memlocation = df['datastore'][num]['parameters']['root']
    pool = os.path.join(memlocation, memName)
    memdata = np.loadtxt(pool)
    memoryArray = np.append(memoryArray, memdata)
    print memoryArray
    print ' '
    print num

[ 4648.]
 
0


In [35]:
df['memory']=memoryArray
columns.append('memory')
df[columns]

,label,N,steps,tags,duration,sweeps,memory
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0


In [38]:
data = {'a' : [1, 2, 3], 'b' : [4, 5, 6]}
pandas.DataFrame(data)

,a,b
0,1,4
1,2,5
2,3,6


In [59]:
data = [{'a' : 1, 'b' : 4}, {'a' : 2, 'b' : 5}, {'a' : 3 ,'b' : 6}]
aa  = pandas.DataFrame(data)

In [60]:
def recalc(row):
    return row.b * 2

In [61]:
print aa

   a  b
0  1  4
1  2  5
2  3  6


In [ ]:
aa['c'] = [recalc(row) for index, row in aa.iterrows()]
print aa

In [73]:
times = np.array([0., 1.0, 1.5])
values = times**2


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(times, values)

In [ ]:
import numpy as np
np.linspace(0., 100., 1000)

In [83]:
!ls Data/1de38ff12ec7/
import glob
glob.glob('Data/1de38ff12ec7/*.npz')

1a10x1step0.npz  1a10x1step10.npz  1a10x1step20.npz  memory_usage.txt


['Data/1de38ff12ec7/1a10x1step0.npz',
 'Data/1de38ff12ec7/1a10x1step10.npz',
 'Data/1de38ff12ec7/1a10x1step20.npz']